In [1]:
### Benchmarks for the 'generate_moves' function

In [ ]:
from guard_towers import Board, BOARD_SIZE, Piece
from time import time
import pandas as pd

In [9]:
from timeit import timeit
from guard_towers import Board, Piece, BOARD_SIZE   # adjust import
from typing import Dict

def empty_board() -> Board:
    """Start with a completely blank board (no pieces)."""
    b = Board()
    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            b.grid[y][x] = None
    return b

def board_from_dict(pieces: Dict[str, Piece]) -> Board:
    """Create a board from a {square: Piece(...)} mapping."""
    b = empty_board()
    for sq, pc in pieces.items():
        b.place(sq, pc)
    return b

def time_generate_moves(board: Board, player: str, repeats: int = 10_000) -> float:
    """Return time (seconds) to call generate_moves `repeats` times."""
    return timeit(lambda: board.generate_moves(player), number=repeats)


In [ ]:
initial_board = Board()
print("Blue to move:", time_generate_moves(initial_board, 'b'))
print("Red  to move:", time_generate_moves(initial_board, 'r'))


Blue to move: 0.1319388750125654
Red  to move: 0.10897666699020192


In [11]:
mid_game_pieces = {
    # Blue
    'D3': Piece('b', 'guard'),
    'E3': Piece('b', 'tower', 3),
    'B4': Piece('b', 'tower', 2),
    'G1': Piece('b', 'tower', 1),

    # Red
    'D6': Piece('r', 'guard'),
    'C5': Piece('r', 'tower', 2),
    'F6': Piece('r', 'tower', 4),
    'A7': Piece('r', 'tower', 1),
}
mid_game_board = board_from_dict(mid_game_pieces)

print("Blue to move:", time_generate_moves(mid_game_board, 'b'))
print("Red  to move:", time_generate_moves(mid_game_board, 'r'))


Blue to move: 0.13726724998559803
Red  to move: 0.10538349999114871


In [12]:
end_game_pieces = {
    # Blue
    'D2': Piece('b', 'guard'),
    'D3': Piece('b', 'tower', 4),

    # Red
    'D7': Piece('r', 'guard'),
    'D5': Piece('r', 'tower', 3),
}
end_game_board = board_from_dict(end_game_pieces)

print("Blue to move:", time_generate_moves(end_game_board, 'b'))
print("Red  to move:", time_generate_moves(end_game_board, 'r'))


Blue to move: 0.09080900001572445
Red  to move: 0.0646641249768436


### Benchmark eval function

In [2]:
import time
from guard_towers import fen_to_board, Board
from evaluation import evaluate

def benchmark_evaluate(fen: str, player: str, iterations: int = 10_000):
    board, player = fen_to_board(fen)

    # Time the evaluation function over multiple iterations
    start = time.time()
    for _ in range(iterations):
        evaluate(board, player)
    end = time.time()

    total_time = end - start
    avg_time = total_time / iterations
    print(f"Evaluated {iterations} times in {total_time:.4f} seconds.")
    print(f"Average time per call: {avg_time:.8f} seconds.")


# print how many moves are possible
def count_moves(board: Board, player: str) -> int:
    """Count the number of possible moves for a player."""
    moves = board.generate_moves(player)
    return len(moves)

fen = 'r3RG5/r16/b16/7/7/7/3BG3 r'
benchmark_evaluate(fen, player='r')
board, player = fen_to_board(fen)
num_moves = count_moves(board, player)
print(f"Number of possible moves for {player}: {num_moves}")

fen = 'b76/3RG3/7/7/7/7/3BG3 b'
benchmark_evaluate(fen, player='b')
board, player = fen_to_board(fen)
num_moves = count_moves(board, player)
print(f"Number of possible moves for {player}: {num_moves}")

Evaluated 10000 times in 0.0292 seconds.
Average time per call: 0.00000292 seconds.
Number of possible moves for r: 6
Evaluated 10000 times in 0.0215 seconds.
Average time per call: 0.00000215 seconds.
Number of possible moves for b: 15


In [1]:
import time
from guard_towers import fen_to_board
from evaluation import find_best_move, transposition_table, pv_table, depth_move_counters

def benchmark_find_best_move_avg(fen: str, player: str, depths: list, runs_per_depth: int = 10):
    board_init, _ = fen_to_board(fen)

    for depth in depths:
        total_time = 0.0
        last_move = None
        for _ in range(runs_per_depth):
            board = board_init.copy()  # fresh board each time

            transposition_table.clear()
            pv_table.clear()
            
            start = time.perf_counter()
            move = find_best_move(board, player, base_depth=depth)
            end = time.perf_counter()
            total_time += (end - start)
            last_move = move  # just for display
        avg_time = total_time / runs_per_depth
        print(f"Depth {depth}: Avg time = {avg_time:.4f}s over {runs_per_depth} runs — Last move: {last_move}")

depth_move_counters.clear()
benchmark_find_best_move_avg('r3RG5/r16/b16/7/7/7/3BG3 r', player='r', depths=[1, 2, 3, 4, 5, 6])
last_key = max(depth_move_counters.keys())
print(depth_move_counters[last_key].items())
# print(depth_move_counters)
depth_move_counters.clear()
benchmark_find_best_move_avg('b76/3RG3/7/7/7/7/3BG3 b', player='b', depths=[1, 2, 3, 4, 5, 6])
last_key = max(depth_move_counters.keys())
print(depth_move_counters[last_key].items())
# print(depth_move_counters)

Depth 1: Avg time = 0.0001s over 10 runs — Last move: A6-A5-1
Depth 2: Avg time = 0.0006s over 10 runs — Last move: A6-A5-1
Depth 3: Avg time = 0.0058s over 10 runs — Last move: A6-A5-1
Depth 4: Avg time = 0.0189s over 10 runs — Last move: A6-A5-1
Depth 5: Avg time = 0.1149s over 10 runs — Last move: A6-A5-1
Depth 6: Avg time = 0.6612s over 10 runs — Last move: A6-A5-1
dict_items([(0, 6), (1, 32), (2, 219), (3, 1218), (4, 10738), (5, 62771)])
Depth 1: Avg time = 0.0002s over 10 runs — Last move: A7-A1-6
Depth 2: Avg time = 0.0007s over 10 runs — Last move: A7-A1-6
Depth 3: Avg time = 0.0099s over 10 runs — Last move: A7-A1-6
Depth 4: Avg time = 0.0384s over 10 runs — Last move: A7-A1-6
Depth 5: Avg time = 0.5765s over 10 runs — Last move: A7-A1-6
Depth 6: Avg time = 2.3710s over 10 runs — Last move: A7-A1-6
dict_items([(0, 15), (1, 60), (2, 945), (3, 3517), (4, 59634), (5, 220917)])
